In [50]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import scipy

In [51]:
path = r'C:\Users\erosb\08-2023 Instacart Basket Analysis'

In [52]:
df = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

### #4 Wrangle data

In [53]:
df.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [54]:
#rename columns
df= df.rename(columns = {'Surnam':'last_name', 'First Name':'first_name'})

In [55]:
df.head()

,user_id,first_name,last_name,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


All columns seem useful to some kind of analysis so none will be dropped.

In [56]:
df.dtypes

user_id          int64
first_name      object
last_name       object
Gender          object
STATE           object
Age              int64
date_joined     object
n_dependants     int64
fam_status      object
income           int64
dtype: object

In [57]:
#change user_id to str
df['user_id'] = df['user_id'].astype('str')

### #5 Consistency checks

In [34]:
df.shape

(206209, 10)

In [28]:
df.describe()

,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000
mean,49.501646,1.499823,94632.852548
std,18.480962,1.118433,42473.786988
min,18.000000,0.000000,25903.000000
25%,33.000000,0.000000,59874.000000
50%,49.000000,1.000000,93547.000000
75%,66.000000,3.000000,124244.000000
max,81.000000,3.000000,593901.000000


No missing values for integer types. Minimum and maximum ages are reasonable. Mean and median are close for age and income suggesting fairly normal distribution. It is surprising that 3 is the max number of dependants - seems like it should be higher.

In [58]:
df['first_name'].value_counts(dropna=False)

NaN        11259
Marilyn     2213
Barbara     2154
Todd        2113
Jeremy      2104
           ...  
Merry        197
Eugene       197
Garry        191
Ned          186
David        186
Name: first_name, Length: 208, dtype: int64

In [59]:
df['last_name'].value_counts(dropna=False)

Hamilton      252
Randall       248
Lamb          243
Pennington    243
Barnett       242
             ... 
Poole         172
Bauer         166
Pearson       164
Payne         163
Jordan        162
Name: last_name, Length: 1000, dtype: int64

11259 unknown first names. Okay - name of customer is not essential for analysis. No unknown last names.

In [60]:
#check for mixed data types
for col in df.columns.tolist():
  weird = (df[[col]].applymap(type) != df[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df[weird]) > 0:
    print (col)

first_name


In [61]:
#make all first names string
df['first_name'] = df['first_name'].astype('str')

In [62]:
#look for missing values
df.isnull().sum()

user_id         0
first_name      0
last_name       0
Gender          0
STATE           0
Age             0
date_joined     0
n_dependants    0
fam_status      0
income          0
dtype: int64

No missing values

In [63]:
#find duplicates
df_dups = df[df.duplicated()]

In [64]:
df_dups

,user_id,first_name,last_name,Gender,STATE,Age,date_joined,n_dependants,fam_status,income


No duplicates

### #6 Combine with other prepared data

In [65]:
df_all = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged4.pkl'))

In [66]:
df.head()

,user_id,first_name,last_name,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [67]:
df_all.head()

,order_id,user_id,order_number,orders_day_of_week,hour_order_placed,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,...,_exist,price_range_loc,busiest_day,busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_flag,med_dspo,frequency_flag
0,2539329,1,1,2,8,NaN,196,1,0,both,...,both,Mid range product,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2398795,1,2,3,7,15.0,196,1,1,both,...,both,Mid range product,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,473747,1,3,3,12,21.0,196,1,1,both,...,both,Mid range product,Slowest days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2254736,1,4,4,7,29.0,196,1,1,both,...,both,Mid range product,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,431534,1,5,4,15,28.0,196,1,1,both,...,both,Mid range product,Slowest days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


Merge on common column "user_id"

In [71]:
df_all.dtypes

order_id                     int64
user_id                      int64
order_number                 int64
orders_day_of_week           int64
hour_order_placed            int64
days_since_prior_order     float64
product_id                   int64
add_to_cart_order            int64
reordered                    int64
_merge                    category
product_name                object
aisle_id                     int64
department_id                int64
prices                     float64
_exist                    category
price_range_loc             object
busiest_day                 object
busiest_period_of_day       object
max_order                    int64
loyalty_flag                object
avg_price                  float64
spending_flag               object
med_dspo                   float64
frequency_flag              object
dtype: object

In [72]:
df_all['user_id'] = df_all['user_id'].astype('str')

In [68]:
#check shapes
df.shape

(206209, 10)

In [69]:
df_all.shape

(32404859, 24)

In [74]:
#merge
df_merged = df.merge(df_all, on='user_id', indicator=False)

In [75]:
df_merged.head()

,user_id,first_name,last_name,Gender,STATE,Age,date_joined,n_dependants,fam_status,income,...,_exist,price_range_loc,busiest_day,busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_flag,med_dspo,frequency_flag
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,both,Mid range product,Busiest days,Most orders,8,New customer,7.988889,Low spender,19.0,Regular customer
1,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,both,Mid range product,Regularly busy,Most orders,8,New customer,7.988889,Low spender,19.0,Regular customer
2,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,both,Mid range product,Busiest days,Most orders,8,New customer,7.988889,Low spender,19.0,Regular customer
3,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,both,Mid range product,Regularly busy,Most orders,8,New customer,7.988889,Low spender,19.0,Regular customer
4,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,both,Mid range product,Slowest days,Most orders,8,New customer,7.988889,Low spender,19.0,Regular customer


In [76]:
df_merged['_merge'].value_counts()

both          32404859
left_only            0
right_only           0
Name: _merge, dtype: int64

Correct

In [77]:
#export
df_merged.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_all.pkl'))